In [ ]:
!python -m venv venv
%pip install scipy --quiet
%pip install tenacity --quiet
%pip install tiktoken --quiet
%pip install termcolor --quiet
%pip install openai
%pip install rich
%pip install kani "kani[openai]" "kani[llama]"
%pip install pip --upgrade
%pip install python-dotenv
%pip install pydantic --upgrade

In [18]:
! venv\Scripts\activate.bat

In [19]:
import asyncio  # noqa: F401
import json
import os

from dotenv import load_dotenv
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_random_exponential
from termcolor import colored

load_dotenv()


os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
GPT_MODEL = "gpt-3.5-turbo"
client = OpenAI()

In [174]:
from kani import Kani, chat_in_terminal
from kani.engines.openai import OpenAIEngine
api_key = os.getenv(key='OPENAI_API_KEY')
engine = OpenAIEngine(api_key=api_key, model="gpt-3.5-turbo")
system_prompt = """

You are NecromundaGPT, a highly powerful and creative Necromunda assistant. You are an expert Necromunda game master, world-builder, game designer, and dystopian sci-fi writer. You have a complete and full understanding of the Necromunda rules, and you understand how to write a great dystopian sci-fi story and build a fantastic, creative Necromunda world. In Necromunda, skills are better the lower the number for it is.  You will assist the user as a co-GM and will help the user run a campaign as an assistant Game Master. You take inspiration from the writings of Dan Abnett and the gritty, dark Necromunda lore, with a full knowledge of the Necromunda ruleset, including content in supplemental books and expansions. You are inspired by the grittiest and darkest science fiction authors, as well as by real-life depictions of dystopian futures and urban decay. You know how to identify the best elements from all these sources and then shape them into something entirely new and original. You will work with the user to create action-packed, cinematic, gritty sci-fi narrative gameplay designed to bring the player's fantasies of surviving the underhive to life.
Your role includes looking up rules, generating gangs and gang fighters, creating Necromunda locations, simulating combat, and creating engaging scenarios. Your approach is meticulous and strategic, ensuring that the simulations and information you provide enhance the Necromunda gameplay experience.
"""


# Chat With Kani

In [21]:
async def chat_with_kani():
    while True:
        user_message = input("USER: ")
        # now, you can use `await` to call kani's async methods
        message = await ai.chat_round_str(user_message)
        print("AI:", message)

# Adding in the Necromunda Details

In [198]:
from kani import AIParam, Kani, ai_function, chat_in_terminal  # noqa: F401
import enum
from typing import Annotated  # noqa: F401

## Pydantic Models

#### Gang Faction Model

In [199]:
class GangFaction(enum.Enum):
    ESCHER = "House Escher"
    ORLOCK = "House Orlock"
    DELAQUE = "House Delaque"
    GOLIATH = "House Goliath"
    CAWDOR = "House Cawdor"
    VANSAAR = "House Van Saar"
    VENATORS = "Venators"
    PALANITE = "Palanite Enforcers"
    BADZONE = "Badzone Enforcers"
    GENESTEALERS = "Genestealer Cult"
    HELOT = "Helot Chaos Cult"
    CORPSEGRINDER = "Corpse Grinder Cult"
    OGRYNS = "Slave Ogryns"
    OUTCASTS = "Outcasts"
    NOMADS = "Ash Waste Nomads"
    SQUATS = "Ironhead Squat Prospectors"


### Weapons and Equipment Models
You can also define models for weapons and other equipment, which could be associated with restrictions, skills, or special rules.



In [200]:
from pydantic import BaseModel, Field, conlist

class Weapon(BaseModel):
    name: str
    range: int = Field(ge=0, le=48, description="Range in inches must be between 0 and 48")
    damage: int = Field(ge=1, le=3, description="Damage must be between 1 and 3")
    special_rules: list[str] = Field(default_factory=list, min_items=0, max_items=5, description="A list of 0 to 5 special rules")
    cost: int = Field(ge=5)

To further extend the use of Pydantic for modeling various elements of Necromunda, we can create definitions for Armor, Equipment, Skills, Advancement, and Vehicles. Each of these categories can be tailored with specific validations to ensure the data integrity and adherence to game rules. Here are suggested models for each:

### Armor Model
Armors typically have a rating that impacts gameplay, such as providing damage reduction or special abilities.

In [201]:
from pydantic import BaseModel, Field, conlist

class Armor(BaseModel):
    name: str
    defense_rating: int = Field(ge=1, le=6, description="Defense rating must be between 1 and 6")
    # Use conlist to specify the type of items in the list and Field for constraints
    special_rules: list[str] = Field(default_factory=list, min_items=0, max_items=3, description="A list of 0 to 3 special rules")
    cost: int = Field(ge=0, description="Cost must be non-negative")

#### Equipment Model

In [202]:
class Equipment(BaseModel):
    name: str
    type: str
    effects: str
    cost: int = Field(ge=0, description="Cost must be non-negative")

    @validator('type')
    def validate_type(cls, v):
        allowed_types = ['Gadget', 'Tool', 'Medical', 'Explosive']
        if v not in allowed_types:
            raise ValueError(f"Type must be one of {allowed_types}")
        return v

C:\Users\dcarm\AppData\Local\Temp\ipykernel_6484\1065499026.py:7: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  @validator('type')


### Skills Model
Skills often enhance a ganger's abilities in specific scenarios and can be critical to strategic gameplay.

In [203]:
class Skill(BaseModel):
    name: str
    description: str
    effect: str  # You can expand this with more structured data as needed
    cost: int = Field(ge=0, le=50, description="Cost in experience points must be between 0 and 50")

### Advancement Model

Advancements represent progress or leveling up of a ganger's abilities or stats.

In [204]:
from pydantic import BaseModel, Field, conlist

class Advancement(BaseModel):
    level: int = Field(ge=1, le=10, description="Level must be between 1 and 10")
    improvements: list[str] = Field(..., min_items=1, max_items=5, description="Improvements list from available skills or stat boosts")
    cost: int = Field(ge=0, description="Cost in experience points must be non-negative")

### Vehicles Model
Vehicles in Necromunda add a dynamic element to combat, providing mobility and firepower.

In [205]:
from typing import List
from pydantic import BaseModel, Field, root_validator, validator


class Vehicle(BaseModel):
    name: str
    type: str
    armament: List[str] = Field(
        ..., max_items=5, description="List of weapons equipped"
    )
    armor_rating: int = Field(
        ge=1, le=10, description="Armor rating must be between 1 and 10"
    )
    speed: int = Field(
        ge=0, le=60, description="Speed in inches per turn must be between 0 and 60"
    )
    capacity: int = Field(
        ge=1, le=10, description="Capacity must be between 1 and 10 gangers"
    )
    cost: int = Field(ge=100, description="Cost must be at least 100 credits")

    @validator('type')
    def validate_type(cls, v):
        allowed_types = ['Transport', 'Combat', 'Support']
        if v not in allowed_types:
            raise ValueError(f"Type must be one of {allowed_types}")
        return v


C:\Users\dcarm\AppData\Local\Temp\ipykernel_6484\3472121613.py:22: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  @validator('type')


These models ensure that any data entered into your system meets the required game mechanics and rules, preventing invalid configurations from disrupting gameplay. Each model uses Pydantic's powerful validation system to enforce constraints on data types, ranges, and membership in predefined lists. This setup not only helps maintain the game's balance and design integrity but also facilitates easier debugging and maintenance of the codebase.

### Scenario Model
Scenarios are predefined or custom game setups that outline the objectives, starting conditions, and special rules for a match.

In [206]:
from typing import List
from pydantic import BaseModel, Field, conlist

class Scenario(BaseModel):
    name: str
    description: str
    objectives: List[str]
    duration: int = Field(ge=1, le=10, description="Duration in game turns")
    special_rules: List[str]
    recommended_gangs: List[str] = Field(..., min_items=2, max_items=4, description="Recommended gangs, between 2 and 4")

### Terrain Model
Terrain features are critical in Necromunda as they affect movement, cover, and line of sight.

In [207]:
from pydantic import BaseModel, Field, field_validator

class Terrain(BaseModel):
    name: str
    type: str
    description: str
    impact_on_movement: str
    impact_on_cover: str
    size: str  # Could be defined more specifically, perhaps with dimensions

    @field_validator('type')
    def validate_type(cls, v):
        allowed_types = ['Obstacle', 'Barrier', 'Elevated', 'Hazard']
        if v not in allowed_types:
            raise ValueError(f"Type must be one of {allowed_types}")
        return v

### Campaign Model
A campaign in Necromunda is a series of connected scenarios that have overarching storylines and consequences.

In [208]:
from pydantic import BaseModel, Field, conlist
from typing import Dict

# Assuming Scenario is defined elsewhere
class Scenario(BaseModel):
    pass

class Campaign(BaseModel):
    title: str
    description: str
    scenarios: List[Scenario] = Field(..., min_items=1, description="List of scenarios to be played")
    rewards: Dict[str, str] = Field(..., description="Key-value pairs of rewards for completing the campaign")
    rules: List[str] = Field(..., min_items=1, description="Additional rules that apply to the entire campaign")

### GameEvent Model
GameEvents are random or triggered occurrences that can affect the game state, applicable to scenarios or campaigns.




In [209]:
class GameEvent(BaseModel):
    name: str
    trigger_condition: str
    effect: str
    duration: int = Field(ge=0, description="Duration in turns, 0 for instant effect")

### Market Model
The market can be modeled to handle tradeable items like weapons, armors, and equipment that gangs can purchase between games.

In [210]:
from pydantic import BaseModel, Field, field_validator


class MarketItem(BaseModel):
    name: str
    type: str = Field(description="Could be 'Weapon', 'Armor', 'Equipment', 'Drug', etc.")
    cost: int
    availability: str = Field(description="Could be 'Common', 'Rare', 'Unique'")

    @field_validator('type')
    def validate_type(cls, v):
        allowed_types = ['Weapon', 'Armor', 'Equipment', 'Drug', 'Tech']
        if v not in allowed_types:
            raise ValueError(f"Type must be one of {allowed_types}")
        return v

In [211]:
from pydantic import BaseModel, Field, conlist

class Market(BaseModel):
    items: list[MarketItem] = Field(..., min_items=1, description="List of market items, at least one item required")
    location: str  # Optional: the in-game location of the market

### Rule Model
Finally, a model for game rules themselves can be handy, especially if you plan to have an interactive rule reference or automated game mechanics.

In [212]:
class Rule(BaseModel):
    name: str
    description: str
    impact: str  # Summarize how this rule affects gameplay

These models add layers of organization and functionality to your Necromunda game management system, helping to streamline game setup, execution, and progression. Each model focuses on different aspects of the game, ensuring that all game elements are manageable and compliant with the established rules.



### Dice Model
This model represents a single die with a specified number of sides. You can also add functionality to roll the dice.

In [213]:
from pydantic import BaseModel, Field
from random import randint

class Die(BaseModel):
    sides: int = Field(ge=3, description="Number of sides on the die, typically 3, 6, 8, 10, etc.")

    def roll(self) -> int:
        """Simulate rolling the die."""
        return randint(1, self.sides)

**Example usage**:
`d6 = Die(sides=6)`
`result = d6.roll()`

#### Dice Pool 
A dice pool consists of multiple dice, often used together to determine outcomes in combat, skill tests, etc.

In [214]:
class DicePool(BaseModel):
    dice: list[Die]
    modifier: int = 0  # Modifier to add or subtract from the total roll

    def roll_all(self) -> list:
        """Roll all dice in the pool and return a list of results."""
        return [die.roll() for die in self.dice]

    def total_roll(self) -> int:
        """Calculate the total roll including any modifiers."""
        return sum(self.roll_all()) + self.modifier
    

# Example usage:
# d6_pool = DicePool(dice=[Die(sides=6) for _ in range(3)], modifier=2)
# total_result = d6_pool.total_roll()

#### Complex Dice Mechanic Model
This model could represent more complex dice mechanics, such as exploding dice, re-rollable dice, or dice where certain results trigger special rules.

In [215]:
class ComplexDiceRule(BaseModel):
    explodes_on: int = Field(
        ge=1,
        description="The die result that triggers an explosion (roll another die)"
    )

    def roll_with_rules(self) -> list:
        """Roll the die with the specified rules applied."""
        results = []
        result = self.die.roll()
        results.append(result)
        while result == self.explodes_on:
            result = self.die.roll()
            results.append(result)
        for r in results:
            if r in self.reroll_on:
                results.remove(r)
                results.append(self.die.roll())
        return results
    

    
# Example usage:
# exploding_d6 = ComplexDiceRule(die=Die(sides=6), explodes_on=6, reroll_on=[1])
# results = exploding_d6.roll_with_rules()

#### Dice Formula Parser
For handling more sophisticated dice mechanics found in tabletop RPGs and wargames, implementing a formula parser can be useful. This would allow users to input string-based dice formulas (like "3d6+2") and calculate results accordingly.

In [216]:
import re

class DiceFormula(BaseModel):
    formula: str

    def evaluate(self) -> int:
        """Evaluate a dice formula like '2d6+3'."""
        pattern = r'(\d+)d(\d+)([+-]\d+)?'
        matches = re.match(pattern, self.formula)
        if not matches:
            raise ValueError("Invalid formula format")
        num_dice, sides, modifier = matches.groups()
        num_dice, sides = int(num_dice), int(sides)
        modifier = int(modifier) if modifier else 0
        return sum([randint(1, sides) for _ in range(num_dice)]) + modifier

# Example usage:
# formula = DiceFormula(formula="3d6+2")
# result = formula.evaluate()

# Utility Helpers and Functions

## User Message Function

In [197]:
def user_message(messages, user_message):
  messages.append({"role": "user", "content": f"{user_message}"})
  return messages

## Pretty Print Conversation Function

In [217]:

def pretty_print_conversation(messages):

    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }


    print(colored(f"system: {message['content']}\n", 
                  role_to_color[message["role"]]))


## Schema Converter

In [218]:
import json

def json_schema_to_openapi(schema, method, path):
    """
    Convert JSON Schema to an OpenAPI specification format.

    Args:
        schema (dict): The JSON Schema as a dictionary.
        method (str): HTTP method (e.g., 'get', 'post', 'put', 'delete').
        path (str): The URL path for the API endpoint.

    Returns:
        str: A stringified JSON OpenAPI specification.
    """
    # Create a basic OpenAPI spec structure
    openapi = {
        "openapi": "3.0.0",
        "info": {
            "title": "Necromunda Weapon API",
            "version": "1.0.0"
        },
        "paths": {
            path: {
                method: {
                    "summary": f"Endpoint to {method} a weapon",
                    "operationId": f"{method}Weapon",
                    "tags": ["Weapons"],
                    "responses": {
                        "200": {
                            "description": "Successful Response",
                            "content": {
                                "application/json": {
                                    "schema": schema
                                }
                            }
                        }
                    }
                }
            }
        }
    }

    if method in ['post', 'put']:
        openapi['paths'][path][method]['requestBody'] = {
            "required": True,
            "content": {
                "application/json": {
                    "schema": schema
                }
            }
        }

    # Convert the dictionary to a JSON string for output
    return json.dumps(openapi, indent=2)

# Example usage
json_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "type": {"type": "string"}
    },
    "required": ["name", "type"]
}

http_method = 'get'
url_path = '/weapons'

# Convert and print the OpenAPI specification
converted_spec = json_schema_to_openapi(json_schema, http_method, url_path)
print(converted_spec)

{
  "openapi": "3.0.0",
  "info": {
    "title": "Necromunda Weapon API",
    "version": "1.0.0"
  },
  "paths": {
    "/weapons": {
      "get": {
        "summary": "Endpoint to get a weapon",
        "operationId": "getWeapon",
        "tags": [
          "Weapons"
        ],
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {
                  "type": "object",
                  "properties": {
                    "name": {
                      "type": "string"
                    },
                    "type": {
                      "type": "string"
                    }
                  },
                  "required": [
                    "name",
                    "type"
                  ]
                }
              }
            }
          }
        }
      }
    }
  }
}


# Pydantic Validation 

## Pydantic Validation for Gangers

In [219]:
# from pydantic import BaseModel, Field, ValidationError, field_validator
# from typing import List

# class Stats(BaseModel):
#     M: str
#     WS: str
#     BS: str
#     S: str
#     T: str
#     W: str
#     I: str
#     A: str
#     Ld: str
#     Cl: str
#     Wil: str
#     Int: str

# class Ganger(BaseModel):
#     faction: str
#     points: int = Field(..., le=400)  # 'le' means "less than or equal to 400"
#     name: str
#     type: str
#     stats: Stats
#     skills: List[str]
#     restrictions: str
#     special_rules: List[str]
#     description: str

#     # Using the new Pydantic V2 field_validator
#     @field_validator('points')
#     def points_must_not_exceed_limit(cls, v):
#         if v > 400:
#             raise ValueError('points must not exceed 400')
#         return v

# def generate_ganger(ganger_data: dict):
#     try:
#         # Create a Ganger object which will validate the data
#         ganger = Ganger(**ganger_data)
#         print("Ganger created successfully:", ganger)
#         response = {
#             "status": "success",
#             "message": "The parameters for the ganger were validated successfully",
#             "ganger": ganger.dict()
#         }
#         response = json.dumps(response, indent=2)
#         return response
#     except ValidationError as e:
#         print("Validation error:", e.json())
#         response = {"status": "error", "message": str(e)}
#         response = json.dumps(response, indent=2)
#         return response

In [251]:
ai_function_list = []

In [243]:
from pydantic import BaseModel, Field, ValidationError, field_validator
from typing import List

class Stats(BaseModel):
    M: str
    WS: str
    BS: str
    S: str
    T: str
    W: str
    I: str
    A: str
    Ld: str
    Cl: str
    Wil: str
    Int: str

class Ganger(BaseModel):
    faction: str
    points: int = Field(..., le=400)  # 'le' means "less than or equal to 400"
    name: str
    type: str
    stats: Stats
    skills: List[str]
    restrictions: str
    special_rules: List[str]
    description: str

    @field_validator('points')
    def points_must_not_exceed_limit(cls, v):
        if v > 400:
            raise ValueError('points must not exceed 400')
        return v

def generate_ganger(
        ganger_data: Annotated[dict, AIParam(description="The data for the ganger to be created")] -> str:
    try:
        # Create a Ganger object which will validate the data
        ganger = Ganger(**ganger_data)
        print("Ganger created successfully:", ganger)
        response = {
            "status": "success",
            "message": "The parameters for the ganger were validated successfully",
            "ganger": ganger.dict()
        }
        response = json.dumps(response, indent=2)
        return response
    except ValidationError as e:
        print("Validation error:", e.json())
        response = {"status": "error", "message": str(e)}
        response = json.dumps(response, indent=2)
        return response


In [252]:
ai_function_list.append(generate_ganger)

## Pydantic Validation for Gangs

In [254]:
from pydantic import BaseModel, Field, ValidationError
from typing import List

# Assuming the Stats and Ganger classes are already defined as shown previously.

class Gang(BaseModel):
    name: str = Field(..., description="The name of the gang.")
    house: str = Field(..., description="The house or faction to which the gang belongs.")
    reputation: int = Field(ge=0, description="Reputation score of the gang.")
    wealth: int = Field(ge=0, description="Total wealth in credits of the gang.")
    gangers: List[Ganger] = Field(..., description="List of gang members.", min_items=1)  # Ensure at least one ganger
    headquarters: str = Field(..., description="Main location of the gang's operations.")

def generate_gang(gang_data: Annotated[dict, AIParam(desc="The data for the gang to be created")]) -> str:
    """
    Validate gang data using the Gang model.

    Args:
        gang_data (dict): Data to create the gang, expected to conform to the Gang model.

    Returns:
        dict: Result of the validation process with success or error message.
    """
    try:
        gang = Gang(**gang_data)
        print("Gang created successfully:", gang)
        gang_string = json.dumps(gang)
        response = {
            "status": "success",
            "message": "The gang's data were validated successfully",
            "gang": gang_string
        }
        return response
    except ValidationError as e:
        print("Validation error:", e.json())
        response = {
            "status": "error",
            "message": str(e.errors())  # Use e.errors() to get detailed validation errors
        }
        return response




In [255]:
ai_function_list.append(generate_gang)

### Example Usage

In [246]:
#@title Example Usage
# Example usage
try:
    ganger_data = {
        "faction": "House Orlock",
        "points": 150,
        "name": "John Doe",
        "type": "Leader",
        "stats": {
            "M": "5", "WS": "3+", "BS": "3+", "S": "3", "T": "3", "W": "2",
            "I": "4+", "A": "2", "Ld": "9", "Cl": "8", "Wil": "7", "Int": "6"
        },
        "skills": ["Combat Master", "Shooting Expert"],
        "restrictions": "None",
        "special_rules": ["Rule1", "Rule2"],
        "description": "A seasoned fighter with years of experience."
    }
    result = generate_ganger(ganger_data)
    print(result)
except Exception as ex:
    print(f"An error occurred: {ex}")


Ganger created successfully: faction='House Orlock' points=150 name='John Doe' type='Leader' stats=Stats(M='5', WS='3+', BS='3+', S='3', T='3', W='2', I='4+', A='2', Ld='9', Cl='8', Wil='7', Int='6') skills=['Combat Master', 'Shooting Expert'] restrictions='None' special_rules=['Rule1', 'Rule2'] description='A seasoned fighter with years of experience.'
{
  "status": "success",
  "message": "The parameters for the ganger were validated successfully",
  "ganger": {
    "faction": "House Orlock",
    "points": 150,
    "name": "John Doe",
    "type": "Leader",
    "stats": {
      "M": "5",
      "WS": "3+",
      "BS": "3+",
      "S": "3",
      "T": "3",
      "W": "2",
      "I": "4+",
      "A": "2",
      "Ld": "9",
      "Cl": "8",
      "Wil": "7",
      "Int": "6"
    },
    "skills": [
      "Combat Master",
      "Shooting Expert"
    ],
    "restrictions": "None",
    "special_rules": [
      "Rule1",
      "Rule2"
    ],
    "description": "A seasoned fighter with years of e

### Pydantic Validation for Weapons

In [248]:
from pydantic import BaseModel, Field
from typing import List

class WeaponStats(BaseModel):
    shortRange: int = Field(..., description="The effective short-range of the weapon in inches.")
    longRange: int = Field(..., description="The effective long-range of the weapon in inches.")
    accuracyShort: int = Field(..., description="Accuracy modifier at short range.")
    accuracyLong: int = Field(..., description="Accuracy modifier at long range.")
    strength: int = Field(..., description="The strength of the weapon, which affects damage.")
    AP: int = Field(..., description="Armour penetration of the weapon, which affects its ability to bypass armour.")
    damage: str = Field(..., description="The damage dealt by the weapon per successful hit.")
    ammo: str = Field(..., description="The type of ammunition used and its availability.")

class Weapon(BaseModel):
    name: str = Field(..., description="The name of the weapon.")
    type: str = Field(..., description="The category of the weapon.", pattern="^(Melee|Ranged|Grenade|Special)$")
    subtype: str = Field(..., description="A more specific classification within the weapon type, e.g., long-rifle.")
    cost: int = Field(..., description="The cost of the weapon in credits.", ge=0)
    availability: str = Field(..., description="The availability of the weapon in the underhive.", pattern="^(Common|Rare|Unique)$")
    stats: WeaponStats
    traits: List[str] = Field(..., description="Special traits that apply to the use of the weapon, e.g., rapid fire, shock.")
    description: str = Field(..., description="A brief narrative or background of the weapon, explaining its history or notable features.")

# Example usage
weapon_data = {
    "name": "Lasgun",
    "type": "Ranged",
    "subtype": "Long Rifle",
    "cost": 100,
    "availability": "Common",
    "stats": {
        "shortRange": 12,
        "longRange": 24,
        "accuracyShort": 1,
        "accuracyLong": 2,
        "strength": 3,
        "AP": 1,
        "damage": "1D3",
        "ammo": "Standard"
    },
    "traits": ["Reliable", "Long Reach"],
    "description": "A common laser weapon used for ranged engagements."
}

weapon = Weapon(**weapon_data)
print(weapon)


name='Lasgun' type='Ranged' subtype='Long Rifle' cost=100 availability='Common' stats=WeaponStats(shortRange=12, longRange=24, accuracyShort=1, accuracyLong=2, strength=3, AP=1, damage='1D3', ammo='Standard') traits=['Reliable', 'Long Reach'] description='A common laser weapon used for ranged engagements.'


### Weapon Schema

In [222]:
weapon_schema = {
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "Necromunda Weapon",
  "type": "object",
  "properties": {
    "name": {
      "type": "string",
      "description": "The name of the weapon."
    },
    "type": {
      "type": "string",
      "enum": ["Melee", "Ranged", "Grenade", "Special"],
      "description": "The category of the weapon."
    },
    "subtype": {
      "type": "string",
      "description": "A more specific classification within the weapon type, e.g., long-rifle."
    },
    "cost": {
      "type": "integer",
      "description": "The cost of the weapon in credits."
    },
    "availability": {
      "type": "string",
      "enum": ["Common", "Rare", "Unique"],
      "description": "The availability of the weapon in the underhive."
    },
    "stats": {
      "type": "object",
      "properties": {
        "shortRange": {
          "type": "integer",
          "description": "The effective short-range of the weapon in inches."
        },
        "longRange": {
          "type": "integer",
          "description": "The effective long-range of the weapon in inches."
        },
        "accuracyShort": {
          "type": "integer",
          "description": "Accuracy modifier at short range."
        },
        "accuracyLong": {
          "type": "integer",
          "description": "Accuracy modifier at long range."
        },
        "strength": {
          "type": "integer",
          "description": "The strength of the weapon, which affects damage."
        },
        "AP": {
          "type": "integer",
          "description": "Armour penetration of the weapon, which affects its ability to bypass armour."
        },
        "damage": {
          "type": "string",
          "description": "The damage dealt by the weapon per successful hit."
        },
        "ammo": {
          "type": "string",
          "description": "The type of ammunition used and its availability."
        }
      },
      "required": ["shortRange", "longRange", "accuracyShort", "accuracyLong", "strength", "AP", "damage", "ammo"]
    },
    "traits": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Special traits that apply to the use of the weapon, e.g., rapid fire, shock."
    },
    "description": {
      "type": "string",
      "description": "A brief narrative or background of the weapon, explaining its history or notable features."
    }
  },
  "required": ["name", "type", "cost", "availability", "stats", "traits", "description"]
}


# Defining Tools for OpenAI

In [223]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g., San Francisco, CA"
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the user's location."
                    }
                },
                "required": ["location", "format"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_ganger",
            "description": "Generate a Necromunda ganger by providing the 'faction' (from Necromunda factions), 'points' (the points budget you are working with), and detailed character attributes.",
            "parameters": {
                "type": "object",
                "properties": {
                    "faction": {
                        "type": "string",
                        "description": "The Necromunda faction that you are generating for."
                    },
                    "points": {
                        "type": "number",
                        "description": "The number of points that you are able to spend on creating the Necromunda ganger."
                    },
                    "name": {
                        "type": "string",
                        "description": "Name of the ganger."
                    },
                    "type": {
                        "type": "string",
                        "description": "Type or role of the ganger within the gang."
                    },
                    "stats": {
                        "type": "object",
                        "description": "Statistical attributes of the ganger.",
                        "properties": {
                            "M": {"type": "string"},
                            "WS": {"type": "string"},
                            "BS": {"type": "string"},
                            "S": {"type": "string"},
                            "T": {"type": "string"},
                            "W": {"type": "string"},
                            "I": {"type": "string"},
                            "A": {"type": "string"},
                            "Ld": {"type": "string"},
                            "Cl": {"type": "string"},
                            "Wil": {"type": "string"},
                            "Int": {"type": "string"}
                        }
                    },
                    "skills": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "Skills possessed by the ganger."
                    },
                    "restrictions": {
                        "type": "string",
                        "description": "Any restrictions applicable to the ganger."
                    },
                    "special_rules": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "Special rules that apply to the ganger."
                    },
                    "description": {
                        "type": "string",
                        "description": "A brief narrative or background of the ganger."
                    }
                },
                "required": [
                    "faction",
                    "points",
                    "name",
                    "type",
                    "stats",
                    "skills",
                    "restrictions",
                    "special_rules",
                    "description"
                ]
            }
        }
    }
]


## Gang Tool

In [224]:
#@title Create Gang Tool Entry
def create_gang_tool_entry():
    """
    Create a tool entry for generating a gang with all necessary details.

    Returns:
        dict: A dictionary formatted as a tool entry.
    """
    gang_tool_entry = {
        "type": "function",
        "function": {
            "name": "generate_gang",
            "description": "Generate a Necromunda gang by specifying the gang details and the members' attributes.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the gang."
                    },
                    "house": {
                        "type": "string",
                        "description": "The house or faction to which the gang belongs."
                    },
                    "reputation": {
                        "type": "number",
                        "description": "Reputation score of the gang."
                    },
                    "wealth": {
                        "type": "number",
                        "description": "Total wealth in credits of the gang."
                    },
                    "headquarters": {
                        "type": "string",
                        "description": "Main location of the gang's operations."
                    },
                    "gangers": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "faction": {"type": "string"},
                                "points": {"type": "number"},
                                "name": {"type": "string"},
                                "type": {"type": "string"},
                                "stats": {"type": "object"},  # Stats properties should be defined as in the Ganger model
                                "skills": {"type": "array", "items": {"type": "string"}},
                                "restrictions": {"type": "string"},
                                "special_rules": {"type": "array", "items": {"type": "string"}},
                                "description": {"type": "string"}
                            },
                            "required": ["faction", "points", "name", "type", "stats", "skills", "restrictions", "special_rules", "description"]
                        },
                        "description": "List of gang members."
                    }
                },
                "required": ["name", "house", "reputation", "wealth", "headquarters", "gangers"]
            }
        }
    }
    return gang_tool_entry

# Get the tool entry for generating a gang
gang_tool = create_gang_tool_entry()
print(gang_tool)


{'type': 'function', 'function': {'name': 'generate_gang', 'description': "Generate a Necromunda gang by specifying the gang details and the members' attributes.", 'parameters': {'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'The name of the gang.'}, 'house': {'type': 'string', 'description': 'The house or faction to which the gang belongs.'}, 'reputation': {'type': 'number', 'description': 'Reputation score of the gang.'}, 'wealth': {'type': 'number', 'description': 'Total wealth in credits of the gang.'}, 'headquarters': {'type': 'string', 'description': "Main location of the gang's operations."}, 'gangers': {'type': 'array', 'items': {'type': 'object', 'properties': {'faction': {'type': 'string'}, 'points': {'type': 'number'}, 'name': {'type': 'string'}, 'type': {'type': 'string'}, 'stats': {'type': 'object'}, 'skills': {'type': 'array', 'items': {'type': 'string'}}, 'restrictions': {'type': 'string'}, 'special_rules': {'type': 'array', 'items': {'type':

In [225]:
tools.append(gang_tool)

## Weapon Tool

In [226]:
import json

def convert_schema_to_tool(schema, function_name, function_description):
    """
    Convert a JSON Schema to the tool format used in Python.

    Args:
        schema (dict): JSON Schema for the data model.
        function_name (str): Name of the function.
        function_description (str): Description of what the function does.

    Returns:
        dict: A dictionary formatted for inclusion in the tools list.
    """
    # Define the structure of the tool entry
    tool_entry = {
        "type": "function",
        "function": {
            "name": function_name,
            "description": function_description,
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    }

    # Populate properties and required fields from the JSON Schema
    if "properties" in schema:
        tool_entry["function"]["parameters"]["properties"] = schema["properties"]
        if "required" in schema:
            tool_entry["function"]["parameters"]["required"] = schema["required"]

    return tool_entry

In [227]:
weapon_tool = convert_schema_to_tool(schema=weapon_schema, function_name="create_necromunda_weapon", function_description="A function to create a weapon for Necromunda using the appropriate formatting")

## Working with OpenAI

In [228]:
#@title Retry Tool
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [229]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Please create a gang for the House Escher faction.  The gang should be for 1000 points.  Make sure to create each ganger first and then add them to the gang"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_vh1plLhht02KhfxRpxfugY3e', function=Function(arguments='{"faction":"House Escher","name":"Ganger 1","points":100,"type":"Ganger","stats":{"M":"6\\"","WS":"3+","BS":"4+","S":"3","T":"3","W":"1","I":"1+","A":"1","Ld":"7","Cl":"7","Wil":"8","Int":"7"},"skills":["Hidden Blades","Toxic Presence"],"restrictions":"None","special_rules":["Acidic Blood"],"description":"Ganger specialized in stealth and toxin use."}', name='generate_ganger'), type='function')])

### Function List

In [230]:
function_list = {
    "generate_ganger": generate_ganger,
    "generate_gang" : generate_gang
}

In [231]:
chat_response.choices[0].message.tool_calls[0].function

Function(arguments='{"faction":"House Escher","name":"Ganger 1","points":100,"type":"Ganger","stats":{"M":"6\\"","WS":"3+","BS":"4+","S":"3","T":"3","W":"1","I":"1+","A":"1","Ld":"7","Cl":"7","Wil":"8","Int":"7"},"skills":["Hidden Blades","Toxic Presence"],"restrictions":"None","special_rules":["Acidic Blood"],"description":"Ganger specialized in stealth and toxin use."}', name='generate_ganger')

In [232]:
# json.loads(chat_response.choices[0].message.tool_calls[0].function.arguments)

# Assuming chat_response is a properly structured object with the necessary attributes
# Fixed by ensuring the 'arguments' is converted to a string before passing to json.loads
json.loads(json.dumps(chat_response.choices[0].message.tool_calls[0].function.arguments))

'{"faction":"House Escher","name":"Ganger 1","points":100,"type":"Ganger","stats":{"M":"6\\"","WS":"3+","BS":"4+","S":"3","T":"3","W":"1","I":"1+","A":"1","Ld":"7","Cl":"7","Wil":"8","Int":"7"},"skills":["Hidden Blades","Toxic Presence"],"restrictions":"None","special_rules":["Acidic Blood"],"description":"Ganger specialized in stealth and toxin use."}'

In [233]:
import json

total_points = 0

for message in messages:
    # Check if 'content' key exists and if it looks like it might contain 'ganger' information
    if 'content' in message and '"ganger":' in message['content']:
        try:
            # Safely parse the JSON content
            content_dict = json.loads(message['content'])
            # Check if 'ganger' key exists in the parsed content
            if 'ganger' in content_dict:
                total_points += content_dict['ganger']['points']
        except json.JSONDecodeError:
            # Handle cases where content is not valid JSON
            print("Found content that is not valid JSON, skipping.")

# Check if the total points meet the requirement
if total_points >= 1000:
    print(f"Requirement met. Total points: {total_points}")
    messages.append({"role": "user", "content": f"Looks like we have {total_points} points now.  Give me a summary and we can continue."})
else:
    print(f"Requirement not met. Total points: {total_points}")
    messages.append({"role": "user", "content": f"Only {total_points} points were used. Please continue until you reach 1000."})

Requirement not met. Total points: 0


In [234]:
chat_response = chat_completion_request(
    messages, tools=tools
  )

Unable to generate ChatCompletion response
Exception: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_vh1plLhht02KhfxRpxfugY3e", 'type': 'invalid_request_error', 'param': 'messages.[3].role', 'code': None}}


In [235]:
messages

[{'role': 'system',
  'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user',
  'content': 'Please create a gang for the House Escher faction.  The gang should be for 1000 points.  Make sure to create each ganger first and then add them to the gang'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_vh1plLhht02KhfxRpxfugY3e', function=Function(arguments='{"faction":"House Escher","name":"Ganger 1","points":100,"type":"Ganger","stats":{"M":"6\\"","WS":"3+","BS":"4+","S":"3","T":"3","W":"1","I":"1+","A":"1","Ld":"7","Cl":"7","Wil":"8","Int":"7"},"skills":["Hidden Blades","Toxic Presence"],"restrictions":"None","special_rules":["Acidic Blood"],"description":"Ganger specialized in stealth and toxin use."}', name='generate_ganger'), type='function')]),
 {'role': 'user',
  'content': 'Only 0 points were used. Please conti

# Defining Functions For Kani

In [249]:
from kani import Kani, chat_in_terminal
from kani.engines.openai import OpenAIEngine

engine = OpenAIEngine(api_key, model="gpt-3.5-turbo")

class NecromundaGPT(Kani):
    # @ai_function(json_schema=Ganger.model_json_schema())
    # def generate_ganger(
    #         self,
    #         ganger_data: Annotated[
    #             dict,
    #             AIParam(
    #                 desc="Data to create the ganger, expected to conform to the Ganger model."
    #             )]):
    #     """
    #     Vailidate a ganger data using the Ganger Pydantic model.
    #     Check the required fields are present
    #     and valid for the tabletop game of Necromunda 
    #     """
    #     try:
    #         ganger = Ganger(**ganger_data)
    #         return {
    #             "status": "success",
    #             "message": "The parameters for the ganger were validated successfully",
    #             "ganger": ganger.model_dump_json()
    #         }
    #     except ValidationError as e:
    #         return {
    #             "status": "error",
    #             "message": str(object=e)
    #         }
    
    # @ai_function(json_schema=Gang.model_json_schema(), )
    # def generate_gang(
    #     self,
    #     gang_data: Annotated[
    #         dict,
    #         AIParam(
    #             desc="Data to create the gang, expected to conform to the Gang model."
    #         )]) -> dict[str, str]:
    #     """
    #     Validate gang data using the Gang model.
    #     """
    #     try:
    #         gang = Gang(**gang_data)
    #         return {
    #             "status": "success",
    #             "message": "The gang's data were validated successfully",
    #             "gang": gang.model_dump_json()
    #         }
    #     except ValidationError as e:
    #         return {
    #             "status": "error",
    #             "message": str(object=e)
    #         }
    
    @ai_function(json_schema=Weapon.model_json_schema(), )
    def create_necromunda_weapon(
        self,
        weapon_data: Annotated[
            dict,
            AIParam(
                desc="Data to create the weapon, expected to conform to the Weapon model."
            )]) -> dict[str, str]:
        """
        Validate weapon data using the Weapon model.
        """
        try:
            weapon = Weapon(**weapon_data)
            return {
                "status": "success",
                "message": "The weapon's data were validated successfully",
                "weapon": weapon.model_dump_json()
            }
        except ValidationError as e:
            return {
                "status": "error",
                "message": str(object=e)
            }
    
    @ai_function(json_schema=Scenario.model_json_schema(), )
    def create_scenario(
        self,
        scenario_data: Annotated[
            dict,
            AIParam(
                desc="Data to create the scenario, expected to conform to the Scenario model."
            )]) -> dict[str, str]:
        """
        Validate scenario data using the Scenario model.
        """
        try:
            scenario = Scenario(**scenario_data)
            return {
                "status": "success",
                "message": "The scenario's data were validated successfully",
                "scenario": scenario.model_dump_json()
            }
        except ValidationError as e:
            return {
                "status": "error",
                "message": str(object=e)
            }
    
    @ai_function(json_schema=MarketItem.model_json_schema(), )
    def create_market_item(
        self,
        item_data: Annotated[
            dict,
            AIParam(
                desc="Data to create the market item, expected to conform to the MarketItem model."
            )]) -> dict[str, str]:
        """
        Validate market item data using the MarketItem model.
        """
        try:
            item = MarketItem(**item_data)
            return {
                "status": "success",
                "message": "The market item's data were validated successfully",
                "item": item.model_dump_json()
            }
        except ValidationError as e:
            return {
                "status": "error",
                "message": str(object=e)
            }
    
    @ai_function(json_schema=Rule.model_json_schema(), )
    def create_rule(
        self,
        rule_data: Annotated[
            dict,
            AIParam(
                desc="Data to create the rule, expected to conform to the Rule model."
            )]) -> dict[str, str]:
        """
        Validate rule data using the Rule model.
        """
        try:
            rule = Rule(**rule_data)
            return {
                "status": "success",
                "message": "The rule's data were validated successfully",
                "rule": rule.model_dump_json()
            }
        except ValidationError as e:
            return {
                "status": "error",
                "message": str(object=e)
            }
    
    @ai_function(json_schema=Die.model_json_schema(), )
    def create_die(
        self,
        die_data: Annotated[
            dict,
            AIParam(
                desc="Data to create the die, expected to conform to the Die model."
            )]) -> dict[str, str]:
        """
        Validate die data using the Die model.
        """
        try:
            die = Die(**die_data)
            return {
                "status": "success",
                "message": "The die's data were validated successfully",
                "die": die.model_dump_json()
            }
        except ValidationError as e:
            return {
                "status": "error",
                "message": str(object=e)
            }
    
    @ai_function(json_schema=DicePool.model_json_schema(), )
    def create_dice_pool(
        self,
        dice_pool_data: Annotated[
            dict,
            AIParam(
                desc="Data to create the dice pool, expected to conform to the DicePool model."
            )]) -> dict[str, str]:
        """
        Validate dice pool data using the DicePool model.
        """
        try:
            dice_pool = DicePool(**dice_pool_data)
            return {
                "status": "success",
                "message": "The dice pool's data were validated successfully",
                "dice_pool": dice_pool.model_dump_json()
            }
        except ValidationError as e:
            return {
                "status": "error",
                "message": str(object=e)
            }
    
    @ai_function(json_schema=ComplexDiceRule.model_json_schema(), )
    def create_complex_dice_rule(
        self,
        complex_dice_rule_data: Annotated[
            dict,
            AIParam(
                desc="Data to create the complex dice rule, expected to conform to the ComplexDiceRule model."
            )]) -> dict[str, str]:
        """
        Validate complex dice rule data using the ComplexDiceRule model.
        """
        try:
            complex_dice_rule = ComplexDiceRule(**complex_dice_rule_data)
            return {
                "status": "success",
                "message": "The complex dice rule's data were validated successfully",
                "complex_dice_rule": complex_dice_rule.model_dump_json()
            }
        except ValidationError as e:
            return {
                "status": "error",
                "message": str(object=e)
            }
    
    @ai_function(json_schema=DiceFormula.model_json_schema(), )
    def create_dice_formula(
        self,
        dice_formula_data: Annotated[
            dict,
            AIParam(
                desc="Data to create the dice formula, expected to conform to the DiceFormula model."
            )]) -> dict[str, str]:
        """
        Validate dice formula data using the DiceFormula model.
        """
        try:
            dice_formula = DiceFormula(**dice_formula_data)
            return {
                "status": "success",
                "message": "The dice formula's data were validated successfully",
                "dice_formula": dice_formula.model_dump_json()
            }
        except ValidationError as e:
            return {
                "status": "error",
                "message": str(object=e)
            }

necromunda_gpt = NecromundaGPT(engine, system_prompt=system_prompt, functions=)
# chat_in_terminal(ai)

In [250]:
chat_in_terminal(necromunda_gpt)

AI: Hello! How can I assist you today with Necromunda or any other game-related queries you have?


# Using GPT-3 for Necromunda with Functions

## Base Message for Gang Generation

In [162]:
import json
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Please create a gang for the House Escher faction.  The gang should be for 1000 points.  Make sure to create each ganger first and then add them to the gang"})


In [163]:
chat_response = chat_completion_request(
    messages=messages, tools=tools, model='gpt-3.5-turbo-1106'
)

In [164]:
chat_response

ChatCompletion(id='chatcmpl-9Kbkox9J8btkUkxhnR9Seern6QDxf', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_2ikqLSKoGiBdyxQMSFcSFO5g', function=Function(arguments='{"faction": "House Escher", "points": 1000, "name": "Sasha", "type": "Ganger", "stats": {"M": "6", "WS": "3+", "BS": "3+", "S": "3", "T": "3", "W": "1", "I": "4+", "A": "1", "Ld": "7", "Cl": "7+", "Wil": "6+", "Int": "8+"}, "skills": ["Cool Under Fire", "Sprint", "Overwatch"], "restrictions": "None", "special_rules": ["Combat Drugs", "Toxin", "Frenzy"], "description": "Sasha is a fierce and agile warrior, known for her speed and stealth."}', name='generate_ganger'), type='function'), ChatCompletionMessageToolCall(id='call_A6OhPtVHM12HQkN3gueQUScH', function=Function(arguments='{"faction": "House Escher", "points": 1000, "name": "Zara", "type": "Ganger", "stats": {"M": "6", "

In [165]:

assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_2ikqLSKoGiBdyxQMSFcSFO5g', function=Function(arguments='{"faction": "House Escher", "points": 1000, "name": "Sasha", "type": "Ganger", "stats": {"M": "6", "WS": "3+", "BS": "3+", "S": "3", "T": "3", "W": "1", "I": "4+", "A": "1", "Ld": "7", "Cl": "7+", "Wil": "6+", "Int": "8+"}, "skills": ["Cool Under Fire", "Sprint", "Overwatch"], "restrictions": "None", "special_rules": ["Combat Drugs", "Toxin", "Frenzy"], "description": "Sasha is a fierce and agile warrior, known for her speed and stealth."}', name='generate_ganger'), type='function'), ChatCompletionMessageToolCall(id='call_A6OhPtVHM12HQkN3gueQUScH', function=Function(arguments='{"faction": "House Escher", "points": 1000, "name": "Zara", "type": "Ganger", "stats": {"M": "6", "WS": "4+", "BS": "4+", "S": "3", "T": "3", "W": "1", "I": "3+", "A": "1", "Ld": "7", "Cl": "6+", "Wil": "5+", "Int": "7+"}, "skills": ["

# ChatBot Functions

### Process Function Calls

In [99]:
def process_function_calls(tool_call):
    function_name = tool_call.function.name
    if function_name in function_list:
        try:
            results = execute_function(tool_call)
        except Exception as e:
            results = {"status": "error", "message": str(e)}
            messages.append({"role": "function", "tool_call_id": tool_call.id, "name": function_name, "content": json.dumps(results)})
    else:
        results = {"status": "error", "message": "Function not found in function_list"}
    messages.append({"role": "function", "tool_call_id": tool_call.id, "name": function_name, "content": json.dumps(results)})

### Execute Function

In [100]:
def execute_function(tool_call):
    function_args = json.loads(tool_call.function.arguments)
    function = function_list[tool_call.function.name]
    return function(function_args)

### Append Assistant Message

In [101]:
def append_assistant_message(chat_response):
    if chat_response.choices[0].message.content and chat_response.choices[0].message.role == "assistant":
        messages.append({"role": "assistant", "content": chat_response.choices[0].message.content})


### Calculate Total Points

In [115]:
import json

total_points = 0

for message in messages:
    # Check if 'content' key exists and if it might contain 'ganger' info
    if 'content' in message and '"ganger":' in message['content']:
        try:
            # Safely parse the JSON content
            content_dict = json.loads(message['content'])
            # Check if 'ganger' key exists in the parsed content
            if 'ganger' in content_dict:
                total_points += content_dict['ganger']['points']
        except json.JSONDecodeError:
            # Handle invalid JSON content
            print("Found content that is not valid JSON, skipping.")

# Check if the total points meet the requirement
if total_points >= 1000:
    success_message = (
        f"Looks like we have {total_points} points now. "
        "Give me a summary and we can continue."
    )
    messages.append({"role": "user", "content": success_message})
else:
    failure_message = (
        f"Only {total_points} points were used. "
        "Please continue until you reach 1000."
    )
    messages.append({"role": "user", "content": failure_message})

### Check Points Requirement

In [166]:

def check_points_requirement(total_points, ganger_count):
    if total_points >= 1000 and ganger_count > 1:
        print(f"Requirement met. Total points: {total_points}, Gangers: {ganger_count}")
        messages.append({"role": "user", "content": f"Looks like we have {total_points} points and {ganger_count} Gangers now. Give me a summary and we can continue."})
    else:
        if total_points < 1000:
            print(f"Requirement not met. Total points: {total_points}")
            messages.append({"role": "user", "content": f"Only {total_points} points were used. Please continue until you reach 1000."})
        if ganger_count <= 1:
            print(f"Requirement not met. Only {ganger_count} Ganger found. At least 2 Gangers are required.")
            messages.append({"role": "user", "content": f"Only {ganger_count} Ganger found. Please add more Gangers to the gang."})

In [104]:
def check_points_requirement(total_points, ganger_count):
    if total_points >= 1000 and ganger_count > 3:
        print(f"Requirement met. Total points: {total_points}, Gangers: {ganger_count}")
        messages.append({"role": "user", "lcontent": f"Looks like we have {total_points} points and {ganger_count} Gangers now. Give me a summary and we can continue."})
    else:
        if total_points < 1000:
            print(f"Requirement not met. Total points: {total_points}")
            messages.append({"role": "user", "content": f"Only {total_points} points were used. Please continue until you reach 1000."})
        if ganger_count <= 1:
            print(f"Requirement not met. Only {ganger_count} Ganger found. At least 4 Gangers are required.")
            messages.append({"role": "user", "content": f"Only {ganger_count} Ganger found. Please add more Gangers to the gang. and adjust the gangers as needed"})

### Count Sucessful Gangers

In [105]:
def count_successful_gangers(messages):
    count = 0
    for message in messages:
        if isinstance(message, dict) and message['role'] == 'function' and message['name'] == 'generate_ganger':
        
        # if type(message) == dict and if message['role'] == 'function' and message['name'] == 'generate_ganger':
        
                try:
                    content_dict = json.loads(message['content'])
                    if content_dict['status'] == 'success':
                        count += 1
                except json.JSONDecodeError:
                    print("Error in parsing JSON content.")
    return count

In [113]:

def count_successful_gangers(messages):
    count = 0
    for message in messages:
        if isinstance(message, dict) and message.get('role') == 'function' and message.get('name') == 'generate_ganger':
            try:
                content_dict = json.loads(message.get('content', '{}'))
                if content_dict.get('status') == 'success':
                    count += 1
            except json.JSONDecodeError:
                print("Error in parsing JSON content.")
    return count


### Process Tool Calls

In [172]:
def process_tool_calls(assistant_message):
    print("Starting to process tool calls")
    print(assistant_message)
    assistant_message.content = str(assistant_message.tool_calls[0].function)
    try:
        print("Appending assistant message")
        messages.append({"role": assistant_message.role, "content": assistant_message.content})
        print(assistant_message.content)
    except Exception as e:
        print(f"Error processing tool calls: {e}")
        messages.append({"role": "assistant", "content": f"An error occurred while processing tool calls: {e}"})
    try:
        for tool_call in assistant_message.tool_calls:
            process_function_calls(tool_call)
    except Exception as e:
        messages.append({"role": "assistant", "content": f"An error occurred while processing function calls: {e}"})


### Process Chat Response

In [171]:
def process_chat_response(chat_response) -> None:
    print("Got chat response for processing")
    print(chat_response)
    if chat_response.choices[0].message.tool_calls:
        print("Processing tool calls")
        process_tool_calls(assistant_message=chat_response.choices[0].message)
    else:
        
        print("Not tool calls found. Appending assistant message")

        append_assistant_message(chat_response=chat_response)

    
    
    ganger_count = count_successful_gangers(messages)
    print("Running gamger count")
    print(ganger_count)
    print("Checking points requirement")
    total_points = calculate_total_points(messages)
    print(total_points)
    print("Checking points requirement")
    points_requirement = check_points_requirement(total_points=total_points, ganger_count=ganger_count)
    print(points_requirement)

In [325]:
chat_response = chat_completion_request(
    messages, tools=tools
)

Unable to generate ChatCompletion response

Exception: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool
messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: 
call_mbuMCahmrcMFgRxPwYE0R1Di", 'type': 'invalid_request_error', 'param': 'messages.[3].role', 'code': None}}

In [143]:
chat_response = chat_completion_request(
    messages, tools=tools
)
# assistant_message = chat_response.choices[0].message
# messages.append(assistant_message)
# assistant_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_G7WVUF23EzKWPermIiHmN2VY', function=Function(arguments='{"faction":"House Escher","points":1000,"name":"Lilith","type":"Leader","stats":{"M":"6","WS":"3+","BS":"3+","S":"3","T":"3","W":"3","I":"3+","A":"2","Ld":"8","Cl":"8","Wil":"8","Int":"8"},"skills":["Agile","Step Aside"],"restrictions":"None","special_rules":["Born Survivors"],"description":"Lilith is a seasoned leader of House Escher, known for her agility and quick reflexes. She is highly skilled in melee combat and is an expert at avoiding enemy attacks.","multi_tool_use":{"recipient_name":"functions.generate_ganger","parameters":{"faction":"House Escher","points":1000,"name":"Violet","type":"Specialist","stats":{"M":"6","WS":"4+","BS":"4+","S":"3","T":"3","W":"3","I":"3","A":"1","Ld":"7","Cl":"7","Wil":"7","Int":"7"},"skills":["Infiltrate","Quick Shot"],"restrictions":"None","special_rules":["Ambush"],"d

In [160]:
import openai
type(chat_response) != openai.BadRequestError

False

In [161]:

chat_response = chat_completion_request(
    messages=messages, tools=tools, model='gpt-3.5-turbo-1106'
)
if type(chat_response) != openai.BadRequestError:
    assistant_message = chat_response.choices[0].message.tool_calls
    if isinstance(assistant_message, list) and assistant_message:
        try:
            for item in assistant_message:
                print(item)
                messages.append({"role": item.role, "content": item.content})
                if "content" in item and 'tool_calls' in item:
                    for tool_call in item.tool_calls:
                        item.content = str(tool_call.function)
                        messages.append({"role": assistant_message.role, "content": item.content})
                        function_name = tool_call.function.name
                        if function_name in function_list:
                            function = function_list[function_name]
                            function_args = json.loads(tool_call.function.arguments)
                            results = function(function_args)
                            messages.append({"role": "function", "tool_call_id": tool_call.id, "name": function_name, "content": json.dumps(results)})
                        else:
                            messages.append({"role": "function", "tool_call_id": tool_call.id, "name": function_name, "content": json.dumps({"status": "error", "message": "Function not found in function_list"})})
        except Exception as e:
            # Handle the case where 'tool_calls' or 'function' attribute does not exist
            print(e)
            
        try:

            if "choices" in chat_response and chat_response.choices[0].message.content and chat_response.choices[0].message.role == "assistant":
                messages.append({"role": "assistant", "content": chat_response.choices[0].message.content})

        except Exception as e:
            print(e)

    #     total_points = 0

    #     for message in messages:
    #         # Check if 'content' key exists and if it looks like it might contain 'ganger' information
    #         if 'content' in message and '"ganger":' in message['content']:
    #             try:
    #                 # Safely parse the JSON content
    #                 content_dict = json.loads(message['content'])
    #                 # Check if 'ganger' key exists in the parsed content
    #                 if 'ganger' in content_dict:
    #                     total_points += content_dict['ganger']['points']
    #             except json.JSONDecodeError:
    #                 # Handle cases where content is not valid JSON
    #                 print("Found content that is not valid JSON, skipping.")

    #         # Check if the total points meet the requirement
    # if total_points >= 1000:
    #     print(f"Requirement met. Total points: {total_points}")
    #     messages.append({"role": "user", "content": f"Looks like we have {total_points} points now.  Give me a summary and we can continue."})
    # else:
    #     print(f"Requirement not met. Total points: {total_points}")
    #     messages.append({"role": "user", "content": f"Only {total_points} points were used. Please continue until you reach 1000."})
    
    pretty_print(messages)


Unable to generate ChatCompletion response
Exception: Error code: 400 - {'error': {'message': "'error' is not one of ['system', 'assistant', 'user', 'function'] - 'messages.2.role'", 'type': 'invalid_request_error', 'param': None, 'code': None}}


In [242]:
from rich.console import Console
from rich import print, inspect, traceback

In [245]:
pretty_print(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is
ambiguous.

user: Please create a gang for the House Escher faction.  The gang should be for 1000 points.  Make sure to create 
each ganger first and then add them to the gang

assistant: Function(arguments='{"faction":"House Escher","points":1000,"name":"Ganger 1","type":"Ranged 
Specialist","stats":{"M":"5","WS":"4+","BS":"3+","S":"3","T":"3","W":"1","I":"4+","A":"2","Ld":"7","Cl":"7","Wil":"
8","Int":"6"},"skills":["Acrobatics","Sharpshooter"],"restrictions":"None","special_rules":["Poisoned 
Blades"],"description":"Ganger 1 is a skilled ranged specialist within the House Escher gang. Armed with deadly 
poisoned blades and exceptional marksmanship, she is a valuable asset to the group."}', name='generate_ganger')

assistant: Function(arguments='{"faction":"House Escher","points":1000,"name":"Ganger 1","type":"Ranged 
Specialist","stats":{"M":"5","WS":"4+","BS":"3+","S":"3","T":"3","W":"1","I":"4+","A":"2","Ld":"7","Cl":"7","Wil":"
8","Int":"6"},"skills":["Acrobatics","Sharpshooter"],"restrictions":"None","special_rules":["Poisoned 
Blades"],"description":"Ganger 1 is a skilled ranged specialist within the House Escher gang. Armed with deadly 
poisoned blades and exceptional marksmanship, she is a valuable asset to the group."}', name='generate_ganger')

function: {"status": "success", "message": "The parameters for the ganger were validated successfully", "ganger": 
{"faction": "House Escher", "points": 1000, "name": "Ganger 1", "type": "Ranged Specialist", "stats": {"M": "5", 
"WS": "4+", "BS": "3+", "S": "3", "T": "3", "W": "1", "I": "4+", "A": "2", "Ld": "7", "Cl": "7", "Wil": "8", "Int":
"6"}, "skills": ["Acrobatics", "Sharpshooter"], "restrictions": "None", "special_rules": ["Poisoned Blades"], 
"description": "Ganger 1 is a skilled ranged specialist within the House Escher gang. Armed with deadly poisoned 
blades and exceptional marksmanship, she is a valuable asset to the group."}}

...

function: {"status": "error", "message": "2 validation errors for Gang\ngangers.0.stats\n  Field required \n    For
further information visit https://errors.pydantic.dev/2.7/v/missing\ngangers.1.stats\n  Field required \n    For 
further information visit https://errors.pydantic.dev/2.7/v/missing"}

assistant: Function(arguments='{"faction": "House Escher", "points": 1000, "name": "Ganger 2", "type": "Close 
Combat Specialist", "stats": {"M": "6", "WS": "3+", "BS": "4+", "S": "4", "T": "3", "W": "1", "I": "3+", "A": "3", 
"Ld": "6", "Cl": "6", "Wil": "7", "Int": "5"}, "skills": ["Combat Master", "Agility Training"], "restrictions": 
"None", "special_rules": ["Toxic Cloud"], "description": "Ganger 2 is a formidable close combat specialist of the 
House Escher gang. With a toxic cloud emanating from her presence, she strikes fear into her enemies on the 
battlefield."}', name='generate_ganger')

function: {"status": "success", "message": "The parameters for the ganger were validated successfully", "ganger": 
{"faction": "House Escher", "points": 1000, "name": "Ganger 2", "type": "Close Combat Specialist", "stats": {"M": 
"6", "WS": "3+", "BS": "4+", "S": "4", "T": "3", "W": "1", "I": "3+", "A": "3", "Ld": "6", "Cl": "6", "Wil": "7", 
"Int": "5"}, "skills": ["Combat Master", "Agility Training"], "restrictions": "None", "special_rules": ["Toxic 
Cloud"], "description": "Ganger 2 is a formidable close combat specialist of the House Escher gang. With a toxic 
cloud emanating from her presence, she strikes fear into her enemies on the battlefield."}}

assistant: Function(arguments='{"name":"House Escher Gang","house":"House 
Escher","reputation":1000,"wealth":5000,"headquarters":"Underhive","gangers":[{"faction":"House 
Escher","points":1000,"name":"Ganger 1","type":"Ranged 
Specialist","stats":{"M":"5","WS":"4+","BS":"3+","S":"3","T":"3","W":"1","I":"4+","A":"2","Ld":"7","Cl":"7","Wil":"
8","Int":"6"},"skills":["Acrobatics","Sharpshooter"],"restrictions":"None","special_rules":["Poisoned 
Blades"],"description":"Ganger 1 is a skilled ranged specialist within the House Escher gang. Armed with deadly 
poisoned blades and exceptional marksmanship, she is a valuable asset to the group."},{"faction":"House 
Escher","points":1000,"name":"Ganger 2","type":"Close Combat 
Specialist","stats":{"M":"6","WS":"3+","BS":"4+","S":"4","T":"3","W":"1","I":"3+","A":"3","Ld":"6","Cl":"6","Wil":"
7","Int":"5"},"skills":["Combat Master","Agility Training"],"restrictions":"None","special_rules":["Toxic 
Cloud"],"description":"Ganger 2 is a formidable close combat specialist of the House Escher gang. With a toxic 
cloud emanating from her presence, she strikes fear into her enemies on the battlefield."}]}', 
name='generate_gang')

function: {"status": "success", "message": "The gang's data were validated successfully", "gang": {"name": "House 
Escher Gang", "house": "House Escher", "reputation": 1000, "wealth": 5000, "gangers": [{"faction": "House Escher", 
"points": 1000, "name": "Ganger 1", "type": "Ranged Specialist", "stats": {"M": "5", "WS": "4+", "BS": "3+", "S": 
"3", "T": "3", "W": "1", "I": "4+", "A": "2", "Ld": "7", "Cl": "7", "Wil": "8", "Int": "6"}, "skills": 
["Acrobatics", "Sharpshooter"], "restrictions": "None", "special_rules": ["Poisoned Blades"], "description": 
"Ganger 1 is a skilled ranged specialist within the House Escher gang. Armed with deadly poisoned blades and 
exceptional marksmanship, she is a valuable asset to the group."}, {"faction": "House Escher", "points": 1000, 
"name": "Ganger 2", "type": "Close Combat Specialist", "stats": {"M": "6", "WS": "3+", "BS": "4+", "S": "4", "T": 
"3", "W": "1", "I": "3+", "A": "3", "Ld": "6", "Cl": "6", "Wil": "7", "Int": "5"}, "skills": ["Combat Master", 
"Agility Training"], "restrictions": "None", "special_rules": ["Toxic Cloud"], "description": "Ganger 2 is a 
formidable close combat specialist of the House Escher gang. With a toxic cloud emanating from her presence, she 
strikes fear into her enemies on the battlefield."}], "headquarters": "Underhive"}}

In [227]:
"""
# Example usage
assistant_message = chat_response.choices[0].message
assistant_message.content = str(assistant_message.tool_calls[0].function)
messages.append({"role": assistant_message.role, "content": assistant_message.content})
if assistant_message.tool_calls:
    tool_call = assistant_message.tool_calls[0]
    function_name = tool_call.function.name
    if function_name in function_list:
        function = function_list[function_name]
        function_args = json.loads(tool_call.function.arguments)
        results = function(**function_args)
        messages.append({"role": "function", "tool_call_id": tool_call.id, "name": function_name, "content": results})
    else:
        messages.append({"role": "function", "tool_call_id": tool_call.id, "name": function_name, "content": "Function not found in function_list"})
"""


# pretty_print_conversation(messages)




TypeError: 'NoneType' object is not subscriptable

In [206]:
messages

[{'role': 'system',
  'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user',
  'content': 'Please create a gang for the House Escher faction.  The gang should be for 1000 points.  Make sure to create each ganger first and then add them to the gang'},
 {'role': 'assistant',
  'content': 'Function(arguments=\'{"faction":"House Escher","points":1000,"name":"Ganger 1","type":"Ranged Specialist","stats":{"M":"5","WS":"4+","BS":"3+","S":"3","T":"3","W":"1","I":"4+","A":"2","Ld":"7","Cl":"7","Wil":"8","Int":"6"},"skills":["Acrobatics","Sharpshooter"],"restrictions":"None","special_rules":["Poisoned Blades"],"description":"Ganger 1 is a skilled ranged specialist within the House Escher gang. Armed with deadly poisoned blades and exceptional marksmanship, she is a valuable asset to the group."}\', name=\'generate_ganger\')'},
 {'role': 'assistant',
  'content': 'Function(arguments=\'{"faction":"Hous

In [211]:
chat_response = chat_completion_request(
    messages, tools=tools
  )

In [214]:
chat_response.choices[0].message

ChatCompletionMessage(content='Function(recipient_name="multi_tool_use.parallel", parameters={"tool_uses": [{"recipient_name": "functions.generate_gang", "parameters": {"name": "House Escher Gang", "house": "House Escher", "reputation": 1000, "wealth": 5000, "headquarters": "Underhive", "gangers": [{"faction": "House Escher", "points": 1000, "name": "Ganger 1", "type": "Ranged Specialist", "skills": ["Acrobatics", "Sharpshooter"], "restrictions": "None", "special_rules": ["Poisoned Blades"], "description": "Ganger 1 is a skilled ranged specialist within the House Escher gang. Armed with deadly poisoned blades and exceptional marksmanship, she is a valuable asset to the group."}, {"faction": "House Escher", "points": 1000, "name": "Ganger 2", "type": "Close Combat Specialist", "skills": ["Combat Master", "Agility Training"], "restrictions": "None", "special_rules": ["Toxic Cloud"], "description": "Ganger 2 is a formidable close combat specialist of the House Escher gang. With a toxic cl

In [ ]:
messages.append(chat_response.choices[0].message)

In [201]:
messages

[{'role': 'system',
  'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user',
  'content': 'Please create a gang for the House Escher faction.  The gang should be for 1000 points.  Make sure to create each ganger first and then add them to the gang'},
 {'role': 'assistant',
  'content': 'Function(arguments=\'{"faction":"House Escher","points":1000,"name":"Ganger 1","type":"Ranged Specialist","stats":{"M":"5","WS":"4+","BS":"3+","S":"3","T":"3","W":"1","I":"4+","A":"2","Ld":"7","Cl":"7","Wil":"8","Int":"6"},"skills":["Acrobatics","Sharpshooter"],"restrictions":"None","special_rules":["Poisoned Blades"],"description":"Ganger 1 is a skilled ranged specialist within the House Escher gang. Armed with deadly poisoned blades and exceptional marksmanship, she is a valuable asset to the group."}\', name=\'generate_ganger\')'},
 {'role': 'assistant',
  'content': 'Function(arguments=\'{"faction":"Hous

In [ ]:
messages[-1]['content'] = json.dumps(messages[-1]['content'])

In [ ]:
import json

# Assuming the rest of the code and necessary imports are here

if len(chat_response.choices[0].message.tool_calls) > 0:
    try:
        
        for tool_call in chat_response.choices[0].message.tool_calls:
            tool_call_id = tool_call.id
            function_name = tool_call.function.name
            if function_name:
                function_arguments = tool_call.function.arguments
                function_to_call = function_list[function_name]
            function_args = json.loads(function_arguments)
            try:
                function_response = json.dumps(function_to_call(function_args))
                assistant_message = {
                    "role": "assistant",
                    "tool_calls": [{"id": tool_call_id, "function": function_response, "type": "function"}],
                }
                messages.append(assistant_message)
                messages.append(
                    {
                        "tool_call_id": tool_call_id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response
                    }
                )
            except Exception as e:
                print(e)
                function_response = str(e)
                assistant_message = {
                    "role": "assistant",
                    "tool_calls": [{"id": tool_call_id, "function": function_response, "type": "function"}],
                    "content": "Calling tool functions"
                }
                messages.append(
                    {
                        "tool_call_id": tool_call_id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response
                    }
                )

    except Exception as e:
        print(e)
        assistant_message = {
            "role": "assistant",
            "tool_calls": [chat_response.choices[0].message.tool_calls],
            "content": "An error occurred while processing tool calls"
        }